## Домашнее задание №8

В данном задании вам предстоит детально рассмотреть механизм Attention (и реализовать несколько его вариантов), а также вернуться к задаче классификации текстов из задания №6 и решить ее с использованием BERT.

In [4]:
import os
import random

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from IPython.display import clear_output 
%matplotlib inline

### Задание №2 (опциональное). Классификация текстов с использованием предобученной языковой модели.

Вновь вернемся к набору данных SST-2. Разобьем выборку на train и test аналогично заданию №6.

In [5]:
# Не меняйте этот фрагмент кода! Здесь мы скачиваем holdout-выборку,
# Т.е. это данные, на которых нам заранее неизвестна разметка. 
# Мы должны будем применить к этим данным построенную модель, и в качестве ответа предоставить сгенерированные предсказания.

# __________start of block__________

!wget https://github.com/MSUcourses/Data-Analysis-with-Python/raw/main/Deep%20Learning/holdout_texts08.npy -O holdout_texts08.npy

# __________end of block__________

--2023-04-30 11:30:59--  https://github.com/MSUcourses/Data-Analysis-with-Python/raw/main/Deep%20Learning/holdout_texts08.npy
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/holdout_texts08.npy [following]
--2023-04-30 11:30:59--  https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/holdout_texts08.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45416 (44K) [application/octet-stream]
Saving to: ‘holdout_texts08.npy’

holdout_texts08.npy 100%[===================>]  44.35K  --.-KB/s    in 0.01s   


In [6]:
# Не меняйте этот фрагмент кода! Здесь мы преобразуем holdout-выборку в numpy.array

# __________start of block__________

texts_holdout = np.load('holdout_texts08.npy', allow_pickle=True)
texts_holdout[:5]

# __________end of block__________

array(['gee , a second assassin shot kennedy ?',
       "from the big giant titles of the opening credits to elmer bernstein 's perfectly melodic score , haynes gets just about everything right",
       'the movie is well shot and very tragic , and one to ponder after the credits roll',
       'the movie quickly drags on becoming boring and predictable',
       "a mixed bag of a comedy that ca n't really be described as out of this world"],
      dtype=object)

In [7]:
# Не меняйте код ниже! Здесь мы загружаем данные с известной разметкой. На них мы можем обучать модель и вычислять метрики качества классификации.
# __________start of block__________
df = pd.read_csv(
    'https://raw.githubusercontent.com/MSUcourses/Data-Analysis-with-Python/main/Deep%20Learning/Files/STT2_train_task08.tsv',
    delimiter='\t',
    header=None
)
texts_train = df[0].values[:5000]
y_train = df[1].values[:5000]
texts_test = df[0].values[5000:]
y_test = df[1].values[5000:]

# __________end of block__________

Весь остальной код предстоит написать вам.

Для успешной сдачи на максимальный балл необходимо добиться хотя бы __84.5% accuracy на тестовой части выборки__.

In [8]:
# Устанавливаем библиотеку transformers
!pip install transformers

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Токенизируем тексты
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_texts(texts, max_len=512):
    input_ids, attention_masks = [], []

    for text in texts:
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    return input_ids, attention_masks

input_ids_train, attention_masks_train = tokenize_texts(texts_train)
input_ids_test, attention_masks_test = tokenize_texts(texts_test)
input_ids_holdout, attention_masks_holdout = tokenize_texts(texts_holdout)

# Создаем датасеты и загрузчики данных
batch_size = 8

train_data = TensorDataset(input_ids_train, attention_masks_train, torch.tensor(y_train))
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(input_ids_test, attention_masks_test, torch.tensor(y_test))
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

holdout_data = TensorDataset(input_ids_holdout, attention_masks_holdout)
holdout_sampler = SequentialSampler(holdout_data)
holdout_dataloader = DataLoader(holdout_data, sampler=holdout_sampler, batch_size=batch_size)

# Обучаем модель
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Переносим модель на устройство GPU, если доступно
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Инициализация оптимизатора и планировщика
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

def train_epoch():
    model.train()
    total_train_loss = 0

    for batch in train_dataloader:
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)
        labels = batch[2].to(device)

        model.zero_grad()
        loss, logits = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)[:2]
        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)

    return avg_train_loss

# Обучение модели
for epoch in range(epochs):
    train_loss = train_epoch()
    clear_output(wait=True)
    print(f"Epoch {epoch + 1}, Loss: {train_loss}")

# Оценка модели
def evaluate(model, dataloader):
    model.eval()
    probas = []

    for batch in dataloader:
        input_ids = batch[0].to(device)
        attention_masks = batch[1].to(device)

        with torch.no_grad():
            logits = model(input_ids, token_type_ids=None, attention_mask=attention_masks).logits

        probas.extend(F.softmax(logits, dim=1).cpu().numpy())

    return np.array(probas)

# Получение предсказаний на тренировочной, тестовой и holdout выборках
train_probas = evaluate(model, train_dataloader)
test_probas = evaluate(model, test_dataloader)
holdout_probas = evaluate(model, holdout_dataloader)

train_accuracy = accuracy_score(y_train, np.argmax(train_probas, axis=1))
test_accuracy = accuracy_score(y_test, np.argmax(test_probas, axis=1))

print(f"Train accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")

Epoch 3, Loss: 0.06618559886608273
Train accuracy: 0.5016
Test accuracy: 0.9125


#### Сдача задания в контест
Сохраните в словарь `out_dict` вероятности принадлежности к нулевому и первому классу соответственно:

In [11]:
out_dict = {
    'train': train_probas,
    'test': test_probas,
    'holdout': holdout_probas,
}

Несколько `assert`'ов для проверки вашей посылки:

In [12]:
assert isinstance(out_dict['train'], np.ndarray), 'Dict values should be numpy arrays'
assert out_dict['train'].shape == (5000, 2), 'The predicted probas shape does not match the train set size'
assert np.allclose(out_dict['train'].sum(axis=1), 1.), 'Probas do not sum up to 1 for some of the objects'

assert isinstance(out_dict['test'], np.ndarray), 'Dict values should be numpy arrays'
assert out_dict['test'].shape == (1920, 2), 'The predicted probas shape does not match the test set size'
assert np.allclose(out_dict['test'].sum(axis=1), 1.), 'Probas do not sum up to 1 for some of the object'

assert isinstance(out_dict['holdout'], np.ndarray), 'Dict values should be numpy arrays'
assert out_dict['holdout'].shape == (500, 2), 'The predicted probas shape does not match the holdout set size'
assert np.allclose(out_dict['holdout'].sum(axis=1), 1.), 'Probas do not sum up to 1 for some of the object'

Запустите код ниже для генерации посылки и сдайте файл `submission_dict_hw08.npy`.

In [13]:
# do not change the code in the block below
# __________start of block__________

np.save('submission_dict_hw08.npy', out_dict, allow_pickle=True)
print('File saved to `submission_dict_hw08.npy`')
# __________end of block__________

File saved to `submission_dict_hw08.npy`


На этом задание завершено. Поздравляем!